# Cross-Checking with Existing Code

Simply put, we need to see if the stuff we have just coded agrees numerically with what has been done before. So, we're going to first fix a particular kinematic bin and then just plug-and-chug using the "old code" and "new code" respectively.

## (0): Importing Libraries

In [1]:
import numpy as np

## (1): Fixing Kinematic Bins:

In [2]:
# Old-code nomenclature:
M2 = 0.938272 * 0.938272
ALP_INV = 137.0359998
PI = 3.1415926535
RAD = PI / 180.

QQ = 1.8200000524520876
x = 0.3429999947547912
k = 5.75
t = -0.1720000058412552
phi = 22.5

# New-code nomenclature:
_MASS_OF_PROTON_IN_GEV = 0.938272
_ELECTROMAGNETIC_FINE_STRUCTURE_CONSTANT = (1. / 137.0359998)
_PI_DECIMAL = 3.141592653589793238462643383279502884197

squared_Q_momentum_transfer = 1.8200000524520876
x_Bjorken = 0.3429999947547912
lab_kinematics_k = 5.75
squared_hadronic_momentum_transfer_t = -0.1720000058412552
azimuthal_phi = 22.5

## (2): Begin Testing:

### (2.1): Checking $\epsilon^{2}$:

### (2.1.1): $\epsilon^{2}$ | Existing Code:

In [3]:
ee = 4. * M2 * x * x / QQ  # epsilon squared
print(ee)

0.2276325325231178


### (2.1.2): $\epsilon^{2}$ | New Code:

In [4]:
epsilon_squared = (4. * _MASS_OF_PROTON_IN_GEV**2 * x_Bjorken**2) / squared_Q_momentum_transfer
print(epsilon_squared)

0.22763253252311777


### (2.1.3): $\epsilon^{2}$ | Error:

In [5]:
def compute_error(accepted_value, calculated_value):
    error = np.abs(accepted_value - calculated_value) / accepted_value
    return error

compute_error(ee, epsilon_squared)

1.219314976993024e-16

### (2.2): Checking $y$:

### (2.2.1): $y$ | Existing Code:

In [6]:
y = sqrt(QQ) / (sqrt(ee) * k)
print(y)

0.49175715830647937


### (2.2.2): $y$ | New Code:

In [7]:
lepton_energy_fraction_y = sqrt(squared_Q_momentum_transfer) / (sqrt(epsilon_squared) * lab_kinematics_k)
print(lepton_energy_fraction_y)

0.49175715830647937


### (2.2.3): $y$ | Error:

In [8]:
compute_error(y, lepton_energy_fraction_y)

0.0

### (2.3): Checking $\xi$:

### (2.3.1): $\xi$ | Existing Code:

In [9]:
xi = x * (1. + t / 2. / QQ) / (2. - x + x * t / QQ)
print(xi)

0.20115437410625223


### (2.3.2): $\xi$ | New Code:

In [10]:
numerator = (1. + (squared_hadronic_momentum_transfer_t / (2. * squared_Q_momentum_transfer)))
denominator = (2. - x_Bjorken + (x_Bjorken * squared_hadronic_momentum_transfer_t / squared_Q_momentum_transfer))
skewness_parameter = x_Bjorken * numerator / denominator
print(skewness_parameter)

0.20115437410625223


### (2.3.3): $\xi$ | Error:

In [11]:
compute_error(xi, skewness_parameter)

0.0

### (2.4): Checking $\Gamma$ (Prefactor):

### (2.4.1): $\Gamma$ | Existing Code:

In [12]:
Gamma = x * y * y / ALP_INV / ALP_INV / ALP_INV / PI / 8. / QQ / QQ / sqrt(1. + ee)
print(Gamma)

3.4944161283283973e-10


### (2.4.2): $\Gamma$ | New Code:

In [13]:
numerator = _ELECTROMAGNETIC_FINE_STRUCTURE_CONSTANT**3 * lepton_energy_fraction_y**2 * x_Bjorken
denominator = 8. * np.pi * squared_Q_momentum_transfer**2 * sqrt(1 + epsilon_squared)
prefactor = numerator / denominator
print(prefactor)

3.4944161282285183e-10


### (2.4.3): $\Gamma$ | Error:

In [14]:
compute_error(Gamma, prefactor)

2.858244506966018e-11

### (2.5): Checking $t_{\text{min}}$:

### (2.5.1): $t_{\text{min}}$ | Existing Code:

In [15]:
tmin = - QQ * (2. * (1. - x) * (1. - sqrt(1. + ee)) + ee) / (4. * x * (1. - x) + ee)
print(tmin)

-0.13821105657385563


### (2.5.2): $t_{\text{min}}$ | New Code:

In [16]:
one_minus_xb = 1. - x_Bjorken
numerator = (2. * one_minus_xb * (1. - sqrt(1. + epsilon_squared))) + epsilon_squared
denominator = (4. * x_Bjorken * one_minus_xb) + epsilon_squared
squared_hadronic_momentum_transfer_t_minimum = -1. * squared_Q_momentum_transfer * numerator / denominator
print(squared_hadronic_momentum_transfer_t_minimum)

-0.13821105657385557


### (2.5.3): $t_{\text{min}}$ | Error:

In [17]:
compute_error(tmin, squared_hadronic_momentum_transfer_t_minimum)

-4.0164045198217856e-16

### (2.6): Checking $\tilde{K}$:

#### (2.6.1): $\tilde{K}$ | Existing Code:

In [18]:
Ktilde_10 = sqrt(tmin - t) * sqrt((1. - x) * sqrt(1. + ee) + ((t - tmin) * (ee + 4. * x * (1. - x)) / 4. / QQ)) * sqrt(1. - y - y * y * ee / 4.) / sqrt(1. - y + y * y * ee / 4.)
print(Ktilde_10)

0.15209184617030805


#### (2.6.2): $\tilde{K}$ | New Code:

In [19]:
tmin_minus_t = squared_hadronic_momentum_transfer_t_minimum - squared_hadronic_momentum_transfer_t
one_minus_xb = 1. - x_Bjorken
second_root_quantity = (one_minus_xb * sqrt(1. + epsilon_squared)) + ((tmin_minus_t * (epsilon_squared + (4. * one_minus_xb * x_Bjorken))) / (4. * squared_Q_momentum_transfer))
first_root_quantity = sqrt(1. - lepton_energy_fraction_y - lepton_energy_fraction_y**2 * epsilon_squared / 4.)
second_root_quantity = sqrt(1. - lepton_energy_fraction_y + lepton_energy_fraction_y**2 * epsilon_squared / 4.)
k_tilde = sqrt(tmin_minus_t) * sqrt(second_root_quantity) * first_root_quantity / second_root_quantity
print(k_tilde)

0.1520700067174505


#### (2.6.3): $\tilde{K}$ | Error:

In [20]:
compute_error(Ktilde_10, k_tilde)

0.00014359384416372687

### (2.7): Checking $K$:

### (2.7.1): $K$ | Existing Code:

In [21]:
K = sqrt(1. - y + y * y * ee / 4.) * Ktilde_10 / sqrt(QQ)
print(K)

0.08145306294235705


### (2.7.2): $K$ | New Code:

In [22]:
prefactor = sqrt((1. - lepton_energy_fraction_y + (epsilon_squared * lepton_energy_fraction_y**2 / 4.)) / squared_Q_momentum_transfer)
kinematic_k = prefactor * k_tilde
print(kinematic_k)

0.08144136678393026


### (2.7.3): $K$ | Error:

In [23]:
compute_error(K, kinematic_k)

0.0001435938441635796

### (2.8): Checking $k \cdot \Delta$:

### (2.8.1): $k \cdot \Delta$ | Existing Code:

In [24]:
KD = - QQ / (2. * y * (1. + ee)) * (1. + 2. * K * np.cos(PI - (phi * RAD)) - t / QQ * (1. - x * (2. - y) + y * ee / 2.) + y * ee / 2.)  # eq. (29)
print(KD)

-1.4416101302101973


### (2.8.2): $k \cdot \Delta$ | New Code:

In [28]:
def convert_degrees_to_radians(degrees: float) -> float:
    return (degrees * np.pi / 180.)

prefactor = squared_Q_momentum_transfer / (2. * lepton_energy_fraction_y * (1. + epsilon_squared))
phi_dependence = 2. * kinematic_k * np.cos(_PI_DECIMAL - convert_degrees_to_radians(azimuthal_phi))
ratio_delta_to_q_squared = squared_hadronic_momentum_transfer_t / squared_Q_momentum_transfer
bjorken_scaling = x_Bjorken * (2. - lepton_energy_fraction_y)
ratio_y_epsilon = lepton_energy_fraction_y * epsilon_squared / 2.
correction = phi_dependence - (ratio_delta_to_q_squared * (1. - bjorken_scaling + ratio_y_epsilon)) + (ratio_y_epsilon)
in_parentheses = 1. + correction
k_dot_delta_result = -1. * prefactor * in_parentheses
print(k_dot_delta_result)

-1.4416427071894276


### (2.8.3): $k \cdot \Delta$ | Error:

In [29]:
compute_error(KD, k_dot_delta_result)

-2.2597634788715964e-05

### (2.9): Checking $P_{1}$:

### (2.9.1): $P_{1}$ | Existing Code:

In [34]:
P1 = 1. + 2. * KD / QQ
print(P1)

-0.5841869106189472


### (2.9.2): $P_{1}$ | New Code:

In [31]:
p1_propagator = 1. + (2. * (k_dot_delta_result / squared_Q_momentum_transfer))
print(p1_propagator)

-0.5842227094961905


### (2.9.3): $P_{1}$ | Error:

In [33]:
compute_error(P1, p1_propagator)

-6.127983457439132e-05

### (2.10): Checking $P_{2}$:

### (2.10.1): $P_{2}$ | Existing Code:

In [35]:
P2 = t / QQ - 2. * KD / QQ
print(P2)

1.489681415627604


### (2.10.2): $P_{2}$ | New Code:

In [36]:
p2_propagator = (2. * (k_dot_delta_result / squared_Q_momentum_transfer)) + (squared_hadronic_momentum_transfer_t / squared_Q_momentum_transfer)
print(p2_propagator)

1.4897172145048472


### (2.10.3): $P_{2}$ | Error:

In [37]:
compute_error(P2, p2_propagator)

2.403123034744427e-05